**PART 2: GEMMA3  Classifier**

This notebook serves as a parallel to the GPT-4 annotation process, with the key difference being the use of a locally-run language model. Here, we use a Gemma3 (the one with 12b parameters) model via Ollama to classify the same sample of tweets. The goal is to compare the performance and reliability of a local, open-source model against a proprietary API-based model like GPT-4.

First we need to run our model with Ollama. We need to download Ollama and make sure is running our models there. For this reason the code requires to go to the web page by Ollama and donwload the resources.

In the terminal we run Ollama with this as a first step to the experiment:
ollama run gemma3:12b

**Structure:**

Library: Imports libraries.

Data: Loads the same 1000 tweet sample that was used for GPT-4 annotation to make the comparison. A smaller subset of 100 tweets is used for the actual run in this notebook.

Prompt:

Few-Shot: A list of few_shot_examples is created. These examples provide the model with a demonstration of the expected input and output format.

Function build_prompt: This constructs the final prompt by combining instructions (similar to the GPT-4 prompt), the few-shot examples, and the new tweet to be classified.

Classifier function: This functions is called classify_with_ollama_gemma and runs the model Gemma3 with Ollama to classify the tweets similar prompt to GPT. It takes a tweet and the few-shot examples as input and calls the local gemma3:12b model via the ollama.chat function. A raw text ouptut is created by the safe_extract_json helper function. Error handling is implemented to catch and report any JSON parsing failures.

Classification and output: It iterates through the sample of tweets, calling the classification function for each one. The results are in the output to the DF as new columns (polar_score, pole_label).

The final labeled DF is saved to debate_tweets_sample_1000_for_annotation_labeled_ollama.csv.

In [5]:
!pip install ollama

In [6]:
import pandas as pd
from tqdm import tqdm
import json
import time
import re
import ollama

# path to sample df
df_path = 'data/sample/debate_tweets_sample_1000_for_annotation.csv'
df = pd.read_csv(df_path).sample(100, random_state=42).reset_index(drop=True)

# Output columns
polar_scores = []
pole_labels = []
valids = []
comments_list = []

# Prompt template for each tweet
def build_prompt(text):
    return f"""
You are a political communication expert. For the following tweet, return only a valid JSON object with the following fields:

- "polar_score": integer from -7 to +7, or "INV" if not relevant
- "pole_label": "left", "right", "neutral", or "inv"
- "valid": 1 if tweet is about the U.S. political debate, 0 if not
- "comments": short rationale, or "" if unnecessary

Tweet: "{text}"

Strictly return only the JSON object. No explanation or introduction.
"""

# Safe JSON extraction function
def safe_extract_json(raw_response):
    try:
        return json.loads(raw_response)
    except json.JSONDecodeError:
        match = re.search(r"\{.*?\}", raw_response, re.DOTALL)
        if match:
            try:
                return json.loads(match.group())
            except:
                return None
        return None

# Main classification loop
for text in tqdm(df['santext'], desc="Classifying with Ollama"):
    try:
        prompt = build_prompt(text.strip())
        response_obj = ollama.chat(
            model='gemma3:12b',
            messages=[
                {"role": "system", "content": "You are a helpful assistant for political tweet labeling."},
                {"role": "user", "content": prompt}
            ]
        )

        response_text = response_obj['message']['content'].strip()
        print(f"\nMODEL RESPONSE:\n{response_text}")  # Debug print

        result = safe_extract_json(response_text)

        if result:
            polar_scores.append(result.get("polar_score", "INV"))
            pole_labels.append(result.get("pole_label", "inv"))
            valids.append(result.get("valid", 0))
            comments_list.append(result.get("comments", ""))
        else:
            print("Parse error.")
            polar_scores.append("INV")
            pole_labels.append("inv")
            valids.append(0)
            comments_list.append("Parse error")

    except Exception as e:
        print(f"Error: {e}")
        polar_scores.append("INV")
        pole_labels.append("inv")
        valids.append(0)
        comments_list.append("Execution error")

    time.sleep(1)  # Optional delay for pacing

# Add columns to DataFrame
df['polar_score'] = polar_scores
df['pole_label'] = pole_labels
df['valid'] = valids
df['comments'] = comments_list

# Save to labeled file
output_path = df_path.replace(".csv", "_labeled_ollama.csv")
df.to_csv(output_path, index=False)
print(f"\n Labeled file saved to: {output_path}")

Classifying with Ollama:   0%|                          | 0/100 [00:00<?, ?it/s]


MODEL RESPONSE:
```json
{
  "polar_score": -3,
  "pole_label": "right",
  "valid": 1,
  "comments": "Expresses negative opinions on both Biden and Trump, favors Trump based on perceived debate performance."
}
```


Classifying with Ollama:   1%|▏                 | 1/100 [00:07<13:04,  7.92s/it]


MODEL RESPONSE:
```json
{
  "polar_score": -3,
  "pole_label": "left",
  "valid": 1,
  "comments": "Criticizes debate moderators and implies Kamala Harris was unfairly treated."
}
```


Classifying with Ollama:   2%|▎                 | 2/100 [00:13<11:07,  6.81s/it]

...


MODEL RESPONSE:
```json
{
  "polar_score": -5,
  "pole_label": "left",
  "valid": 1,
  "comments": "Directly addresses a claim made by Trump regarding Biden's policy, with a counter-argument referencing Trump's past proposals. Strong partisan framing."
}
```


Classifying with Ollama:  99%|████████████████▊| 99/100 [12:10<00:06,  6.84s/it]


MODEL RESPONSE:
```json
{
  "polar_score": 1,
  "pole_label": "left",
  "valid": 1,
  "comments": "Implies a negative view of Trump, advocating for time limits."
}
```


Classifying with Ollama: 100%|████████████████| 100/100 [12:16<00:00,  7.36s/it]


✅ Done! Labeled file saved to: data/sample/debate_tweets_sample_1000_for_annotation_labeled_ollama.csv
